# Deep Learning-Based WWR and Floor Count Extraction from Façade Images to Improve UBEM

CISBAT 2025

[Ayca Duran](https://systems.arch.ethz.ch/ayca-duran), [Panagiotis Karapiperis](https://www.linkedin.com/in/panagiotis-karapiperis-ethz/), [Christoph Waibel](https://systems.arch.ethz.ch/christoph-waibel), [Arno Schlueter](https://systems.arch.ethz.ch/arno-schlueter)

### Facade Transformation Workflow

This notebook uses the rectification.py file as introduced in [K. Chaudhury, S. DiVerdi and S. Ioffe, "Auto-rectification of user photos" 2014 IEEE International Conference on Image Processing (ICIP), Paris, France, 2014, pp. 3479-3483, doi: 10.1109/ICIP.2014.7025706.](https://ieeexplore.ieee.org/document/7025706) and implemented in [github repo](https://github.com/chsasank/Image-Rectification?tab=readme-ov-file) to automatically rectify images and their facade masks using homography transfomration from vanishing points estimation.

In [ ]:
# Import Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import ransac
from rectification import *
from tqdm import tqdm
from PIL import Image

## Step 1: Facade Simplification

This snipset of the notebook loads the facade predictions from the model and simplifies them into simpler shapes using Open CV approxPolyDP.

In [7]:
# Step 1: Load the image and convert to grayscale
image_path = "example/images"
facade_predictions_path = "example/predictions/facades_roboflow"

# Simplify Facade Predictions
output_path_simplified_facade = "example/predictions/facades_roboflow/facades_simplified"
output_path_compare_simplified_prediction = "example/predictions/facades_roboflow/facades_simplified/figs"
if not os.path.exists(output_path_simplified_facade):
    os.makedirs(output_path_simplified_facade, exist_ok=True)
if not os.path.exists(output_path_compare_simplified_prediction):
    os.makedirs(output_path_compare_simplified_prediction, exist_ok=True)

In [ ]:
# Define output paths
output_path_VP_masked_images = "example/rectified/images"
output_path_VP_masked_annotations = "example/rectified/facades"
output_path_VP_masked_rect_img_annot = "example/rectified/facades/figs"

os.makedirs(output_path_VP_masked_images, exist_ok=True)
os.makedirs(output_path_VP_masked_annotations, exist_ok=True)
os.makedirs(output_path_VP_masked_rect_img_annot, exist_ok=True)

In [ ]:
for filename in tqdm(os.listdir(image_path)):
    image = cv2.imread(os.path.join(image_path, filename))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load facade prediction & Simplify
    facade_prediction = np.array(Image.open(os.path.join(facade_predictions_path, filename.split(".")[0] + ".png")))
    contours, _ = cv2.findContours(facade_prediction, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    epsilon = 0.015 * cv2.arcLength(contours[0], True) 
    approx_contour = cv2.approxPolyDP(contours[0], epsilon, True)

    # Create a blank canvas and draw the simplified contour
    simplified_facade_prediction = np.zeros_like(facade_prediction)
    cv2.drawContours(simplified_facade_prediction, [approx_contour], -1, 1, -1)
    save_filename_facade_simpl = filename.split(".")[0] + "simplified.png"
    cv2.imwrite(os.path.join(output_path_simplified_facade, save_filename_facade_simpl), simplified_facade_prediction)

    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True)
    axes[0].imshow(facade_prediction, cmap="gray")
    axes[0].set_title("Facade Prediction")
    axes[0].grid(True)
    axes[1].imshow(image_rgb)
    axes[1].imshow(simplified_facade_prediction, cmap="rainbow", alpha= 0.5)
    axes[1].set_title("Prediction Simplification")
    axes[1].grid(True)
    plt.savefig(os.path.join(output_path_compare_simplified_prediction, filename), dpi=300, bbox_inches='tight')
    plt.close()
    #break

## Step 2: Auto-Rectify

This code uses a masked region of the image (bounding box of the predicted facade) to estimate vanishing points and perform homography transformation. The auto-rectified images and facade masks are saved under example/rectified.

In [ ]:
# Iterate over the inference images
for filename in tqdm(os.listdir(image_path)):

    image = cv2.imread(os.path.join(image_path, filename))
    height, width, channels = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    simplified_facade_prediction = cv2.imread(os.path.join(output_path_simplified_facade, filename.split(".")[0]+"simplified.png"), cv2.IMREAD_GRAYSCALE)

    ### Crop image with bounding box around facade prediction
    y_indices, x_indices = np.where(simplified_facade_prediction > 0)
    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()

    cropped_facade = image_rgb[max(0, y_min-100):min(height,y_max+1+100), max(0,x_min-100):min(width,x_max+1+100)]
    cropped_facade_annotation = simplified_facade_prediction[max(0,y_min-100):min(height,y_max+1+100), max(0,x_min-100):min(width,x_max+1+100)]

    ### HOMOGRAPHY
    # Compute edgelets
    edgelets1_masked = compute_edgelets(cropped_facade)
    #vis_edgelets(cropped_facade, edgelets1_masked) # Visualize the edgelets
    # Calculate 1st vanishing point
    vp1_masked = ransac_vanishing_point(edgelets1_masked, num_ransac_iter=200, threshold_inlier=5)
    vp1_masked = reestimate_model(vp1_masked, edgelets1_masked, threshold_reestimate=2)
    #vis_model(cropped_facade, vp1_masked) # Visualize the vanishing point model
    # Compute 2nd vanishing point
    edgelets2_masked = remove_inliers(vp1_masked, edgelets1_masked, 10)
    vp2_masked = ransac_vanishing_point(edgelets2_masked, num_ransac_iter=200, threshold_inlier=5)
    vp2_masked = reestimate_model(vp2_masked, edgelets2_masked, threshold_reestimate=2)
    #vis_model(cropped_facade, vp2_masked) # Visualize the vanishing point model

    # Apply Hmography transformation
    warped_img_masked = compute_homography_and_warp(cropped_facade, vp1_masked, vp2_masked, clip_factor=4)
    warped_img_uint8_masked = (warped_img_masked * 255).astype(np.uint8)
    image_to_save_masked = cv2.cvtColor(warped_img_uint8_masked, cv2.COLOR_BGR2RGB)
    save_filename_img_masked = filename.split(".")[0] + "_rectified.jpg"
    cv2.imwrite(os.path.join(output_path_VP_masked_images, save_filename_img_masked), image_to_save_masked)

    warped_annot_masked = compute_homography_and_warp(cropped_facade_annotation, vp1_masked, vp2_masked, clip_factor=4)
    warped_annot_uint8_masked = (warped_annot_masked * 255).astype(np.uint8)
    save_filename_annot_masked = filename.split(".")[0] + "_rectified.png"
    cv2.imwrite(os.path.join(output_path_VP_masked_annotations, save_filename_annot_masked), warped_annot_uint8_masked)

    fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True)
    axes[0].imshow(warped_img_uint8_masked)
    axes[0].set_title("Rectified Image")
    axes[0].grid(False)
    axes[1].imshow(warped_annot_uint8_masked, cmap="rainbow")
    axes[1].set_title("Rectified Annotation")
    axes[1].grid(False)

    ### Find bounding box
    y_indices, x_indices = np.where(warped_annot_masked > 0)
    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()
    # visualize bounding box
    rgb_mask_masked = np.stack([warped_annot_masked] * 3, axis=-1) * 255  # Convert binary to RGB
    cv2.rectangle(rgb_mask_masked, (x_min, y_min), (x_max, y_max), color=(0, 255, 0), thickness=7)
    axes[2].imshow(warped_img_uint8_masked)
    axes[2].imshow(rgb_mask_masked, alpha=0.5)
    axes[2].set_title("Bounding Box")
    axes[2].grid(False)
    plt.savefig(os.path.join(output_path_VP_masked_rect_img_annot, save_filename_img_masked),dpi=300, bbox_inches='tight')
    plt.close()
    #break